In [1]:
!pip install navec

In [2]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar -O navec_hudlit_v1_12B_500K_300d_100q.tar

--2023-10-29 15:41:35--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.4MB/s    in 3.1s    

2023-10-29 15:41:39 (16.4 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [3]:
from navec import Navec

In [4]:
model_navec = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
f = open("/content/drive/MyDrive/datasets/dostoevsky.txt", 'r').read()

In [7]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
words = word_tokenize(f)[:5000]

In [9]:
!pip install -q pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 46.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [10]:
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import pymorphy2

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
def clean_words(x):
    # разбиение по словам
    x = [i.lower() for i in x]

    # удаление знаков пунктуации
    stopPunktual = string.punctuation
    x = [word for word in x if word not in stopPunktual]
    # Лемматизация
    morph = pymorphy2.MorphAnalyzer()
    pymorphy_results = list(map(lambda y: morph.parse(y), x))
    x = ' '.join([res[0].normal_form for res in pymorphy_results])
    x = list(x.split(' '))
    return set(x)

In [12]:
words = clean_words(words)

In [13]:
word_vectors = []
words_2 = words
words = []
for word in words_2:
    try:
        word_vectors.append(model_navec[word.lower()])
        words.append(word)
    except:
        pass

In [14]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(word_vectors)
word_vectors_pca = pca.transform(word_vectors)

In [15]:
word_vectors_pca

array([[-1.81461599, -0.46894872],
       [ 0.17399702, -0.15642989],
       [ 0.97690712,  1.81399546],
       ...,
       [-2.15577426, -0.4334668 ],
       [-1.44487764, -0.84924476],
       [-2.36805418, -0.05111721]])

In [16]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(word_vectors_pca)
word_vectors_pca = ss.transform(word_vectors_pca)

In [17]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig


draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

figure(id='p1004', ...)

In [18]:
import gensim.downloader as api
model_embedding = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


In [19]:
word_vectors = []
words_2 = words
words = []
for word in words_2:
    try:
        word_vectors.append(model_navec[word.lower()])
        words.append(word)
    except:
        pass

In [20]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(word_vectors)
word_vectors_pca = pca.transform(word_vectors)

In [21]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(word_vectors_pca)
word_vectors_pca = ss.transform(word_vectors_pca)

In [22]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig


draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

figure(id='p1049', ...)